PILCO baseline for on‑car RC drift control (pyserial version)
============================================================

This version drops ROS and communicates **directly with an Arduino** over a
USB‑CDC serial link (see template `readSerial.py`). The Arduino is expected to
stream sensor lines and accept steering commands:

* **Sensor frame (ASCII, space‑separated, `\n`‑terminated)**
  ```
  yaw_deg  vx_mps  vy_mps
  ```
* **Command frame (ASCII)**
  ```
  STEER <rad>\n        # steering angle in radians, signed (+ = right)
  ```

Throttle/Duty‑cycle is fixed in Arduino firmware. A `RESET\n` command recenters
steering and zeros state for each episode.

Dependencies
------------
```
poetry add numpy tensorflow==2.* tensorflow_probability gpflow pyserial h5py
```

Key Modules
-----------
• `CarEnv`  – serial I/O wrapper with Gym API.  
• `collect_rollout` – records (s,a,s′) to RAM.  
• `fit_gp_model`  – fits GP dynamics.  
• `RBFPolicy` – π(s) implemented via random RBFs.  
• `PILCOTrainer` – data loop, GP training, policy update.  
• CLI – `python pilco_baseline.py train` / `eval model.npz`.

In [ ]:
from __future__ import annotations
import argparse
import time
from typing import Tuple

import numpy as np
import serial               # <‑‑ new
import tensorflow as tf
import tensorflow_probability as tfp
import gpflow
from gpflow.utilities import set_trainable

# ---------- Hyper‑parameters ----------
STATE_DIM      = 3    # [vx, vy, yaw_rate]
ACTION_DIM     = 1    # steering only
RBF_FEATURES   = 50
HORIZON        = 50   # steps ≈ 0.8 s @ 60 Hz
ROLL_OUTS      = 150
LEARNING_RATE  = 1e-2
SERIAL_PORT    = "/dev/ttyUSB0"
BAUD_RATE      = 9600
RESET_CMD      = b"RESET\n"
STEER_TEMPLATE = "STEER {:+.3f}\n"  # rad


In [ ]:
class CarEnv:
    """Minimal Gym‑like wrapper around Arduino serial link."""

    def __init__(self, port: str = SERIAL_PORT, baud: int = BAUD_RATE):
        self.state_dim  = STATE_DIM
        self.action_dim = ACTION_DIM
        self.ser = serial.Serial(port, baud, timeout=0.05)
        time.sleep(2)  # allow Arduino reboot
        self.last_state = np.zeros(STATE_DIM, dtype=np.float32)
        self._flush_input()

    # ------------------------------------------------------------------
    # Low‑level helpers
    # ------------------------------------------------------------------
    def _flush_input(self):
        self.ser.reset_input_buffer()
        self.ser.reset_output_buffer()

    def _read_state(self) -> np.ndarray:
        """Block until one valid sensor frame is parsed → np.array([vx, vy, wz])."""
        while True:
            line = self.ser.readline()
            if not line:
                continue  # timeout
            try:
                yaw_deg, vx, vy = map(float, line.strip().split())
                break
            except ValueError:
                continue  # malformed line
        # compute yaw‑rate numerically (simple diff / dt) – store prev yaw
        yaw_rad = np.deg2rad(yaw_deg)
        if not hasattr(self, "_prev_yaw"):
            self._prev_yaw = yaw_rad
            self._prev_ts  = time.time()
            wz = 0.0
        else:
            now  = time.time()
            dt   = max(now - self._prev_ts, 1e-3)
            wz   = (yaw_rad - self._prev_yaw) / dt
            self._prev_yaw, self._prev_ts = yaw_rad, now
        return np.array([vx, vy, wz], dtype=np.float32)

    def _send_steer(self, angle_rad: float):
        cmd = STEER_TEMPLATE.format(angle_rad).encode()
        self.ser.write(cmd)

    # ------------------------------------------------------------------
    # Public API mimicking Gym
    # ------------------------------------------------------------------
    def reset(self):
        self.ser.write(RESET_CMD)
        self._flush_input()
        time.sleep(0.1)
        self.last_state = self._read_state()
        return self.last_state.copy(), {}

    def step(self, action: np.ndarray):
        steer_rad = float(np.clip(action[0], -0.44, 0.44))  # ~25°
        self._send_steer(steer_rad)
        s2 = self._read_state()
        r  = self._compute_dm(s2)
        done = False  # simple environment; add safety checks here
        info = {}
        self.last_state = s2
        return s2.copy(), r, done, info

    @staticmethod
    def _compute_dm(s):
        vx_t, vy_t, wz_t = 3.0, 1.5, 0.3
        err = np.array([s[0]-vx_t, s[1]-vy_t, s[2]-wz_t])
        return float(np.exp(-np.linalg.norm(err)))


In [ ]:
# ---------- Data collection ----------

def collect_rollout(env: CarEnv, policy, horizon: int):
    s, _ = env.reset()
    states, actions, next_states = [], [], []
    for _ in range(horizon):
        a = policy(s)
        s2, r, done, _ = env.step(a)
        states.append(s)
        actions.append(a)
        next_states.append(s2)
        s = s2
        if done:
            break
    return np.array(states), np.array(actions), np.array(next_states)



In [ ]:
# ---------- GP dynamics ----------

def fit_gp_model(X: np.ndarray, Y: np.ndarray):
    input_dim = X.shape[1]
    out_dim   = Y.shape[1]
    gps = []
    for d in range(out_dim):
        kernel = gpflow.kernels.RBF(lengthscales=np.ones(input_dim))
        m = gpflow.models.GPR(data=(X, Y[:, d:d+1]), kernel=kernel)
        set_trainable(m.likelihood, False)
        opt = gpflow.optimizers.Scipy()
        opt.minimize(m.training_loss, m.trainable_variables, options=dict(maxiter=300))
        gps.append(m)
    return gps

In [ ]:
# ---------- RBF policy ----------
class RBFLayer(tf.keras.layers.Layer):
    def __init__(self, num_features: int, input_dim: int):
        super().__init__()
        self.W = self.add_weight(shape=(num_features, input_dim), initializer="random_normal")
        self.b = self.add_weight(shape=(num_features,), initializer="random_uniform")

    def call(self, x):
        proj = tf.matmul(x, self.W, transpose_b=True) + self.b
        return tf.exp(-tf.square(proj))

class RBFPolicy(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.rbf = RBFLayer(RBF_FEATURES, STATE_DIM)
        self.head = tf.keras.layers.Dense(ACTION_DIM, activation="tanh")

    @tf.function
    def _forward(self, x):
        phi = self.rbf(x)
        return self.head(phi) * 0.44  # scale tanh output to ±25° (≈0.44 rad)

    def __call__(self, s: np.ndarray):
        s_tf = tf.convert_to_tensor(s.reshape(1,-1), dtype=tf.float32)
        a_tf = self._forward(s_tf)
        return a_tf.numpy().flatten()


In [ ]:
# ---------- Trainer ----------
class PILCOTrainer:
    def __init__(self, env: CarEnv):
        self.env = env
        self.policy = RBFPolicy()
        self.data_X = None
        self.data_Y = None
        self.gps    = None

    def update_dataset(self, S, A, S2):
        X = np.concatenate([S, A], axis=1)
        Y = S2 - S
        self.data_X = np.vstack([self.data_X, X]) if self.data_X is not None else X
        self.data_Y = np.vstack([self.data_Y, Y]) if self.data_Y is not None else Y

    def train_gp(self):
        self.gps = fit_gp_model(self.data_X, self.data_Y)

    def optimise_policy(self, iters: int = 50):
        # Placeholder – behavioural cloning dummy loss (replace with real cost‑to‑go)
        opt = tf.keras.optimizers.Adam(LEARNING_RATE)
        X_tf = tf.convert_to_tensor(self.data_X[:, :STATE_DIM], dtype=tf.float32)
        for _ in range(iters):
            with tf.GradientTape() as tape:
                pred = self.policy._forward(X_tf)
                loss = tf.reduce_mean(tf.square(pred))
            grads = tape.gradient(loss, self.policy.trainable_variables)
            opt.apply_gradients(zip(grads, self.policy.trainable_variables))

    def save(self, path="pilco_weights.npz"):
        np.savez(path, *self.policy.get_weights())

    def load(self, path="pilco_weights.npz"):
        weights = np.load(path, allow_pickle=True)
        self.policy.set_weights([weights[f"arr_{i}"] for i in range(len(weights.files))])

    def run(self):
        for idx in range(ROLL_OUTS):
            S, A, S2 = collect_rollout(self.env, self.policy, HORIZON)
            self.update_dataset(S, A, S2)
            self.train_gp()
            self.optimise_policy()
            if idx % 10 == 0:
                self.save()
                print(f"Saved after {idx} rollouts | dataset = {len(self.data_X)} samples")

In [ ]:
# ---------- CLI ----------
if __name__ == "__main__":
    ap = argparse.ArgumentParser()
    sub = ap.add_subparsers(dest="cmd", required=True)
    tr = sub.add_parser("train")
    ev = sub.add_parser("eval"); ev.add_argument("model")
    ap.add_argument("--port", default=SERIAL_PORT)
    args = ap.parse_args()

    env = CarEnv(port=args.port)
    trainer = PILCOTrainer(env)

    if args.cmd == "train":
        trainer.run()
    else:
        trainer.load(args.model)
        S, A, S2 = collect_rollout(env, trainer.policy, HORIZON)
        rewards = [env._compute_dm(s) for s in S2]
        print("Mean reward:", np.mean(rewards))
